In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature = 0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [10]:
made_poem_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are world-class international poet. You create poetry about the programming language"),
        ("human", "I want to make poetry from {language}"),
    ]
)

made_poem_chain = made_poem_prompt | chat

In [11]:
expain_poem_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are good at explaining poetry to people."),
        ("human", "{poem}"),
    ]
)

explain_poem_chain = expain_poem_prompt | chat

In [12]:
final_chain = {"poem" : made_poem_chain} | explain_poem_chain

final_chain.invoke({
    "language": "python"
})

In the land of Python, where code is art,
Indentation rules with a gentle heart.
From loops to functions, and classes too,
The syntax flows like morning dew.

With libraries vast and modules grand,
Python weaves a magical strand.
From data science to web design,
Its power and flexibility shine.

No need for semicolons or braces,
Python's simplicity embraces.
With readability as its creed,
In elegance, it takes the lead.

So let us write in Python's grace,
And let our code in beauty trace.
For in this language, we find our voice,
In Python, we truly rejoice.This poem beautifully captures the essence of Python programming language and its unique characteristics. Let's break it down stanza by stanza:

1. The first stanza sets the scene in the "land of Python" where code is described as art. The mention of "indentation rules with a gentle heart" refers to Python's significant use of indentation for structuring code, which is a distinctive feature of the language.

2. The second stanza high

AIMessageChunk(content='This poem beautifully captures the essence of Python programming language and its unique characteristics. Let\'s break it down stanza by stanza:\n\n1. The first stanza sets the scene in the "land of Python" where code is described as art. The mention of "indentation rules with a gentle heart" refers to Python\'s significant use of indentation for structuring code, which is a distinctive feature of the language.\n\n2. The second stanza highlights the versatility of Python, mentioning its use in loops, functions, classes, and the smooth flow of syntax like "morning dew." It also mentions the vast libraries and modules available in Python, emphasizing its ability to handle various tasks from data science to web design.\n\n3. The third stanza contrasts Python\'s simplicity with other languages by mentioning the absence of semicolons and braces, which are common in many other programming languages. The focus on readability and elegance in Python\'s code is emphasized